In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.DataFrame([[1, 'a'], [2, 'b'], [3, 'c'], [4, 'd']],
                  columns = ['число', 'буква'],
                 )
df

In [4]:
data = pd.read_csv(r"S02-hw-dataset.csv")

In [5]:
data.head(10)

,user_id,age,country,purchases,revenue
0,1,25.0,FR,7,749
1,2,24.0,RU,5,1115
2,3,52.0,FR,7,399
3,4,31.0,RU,6,654
4,5,NaN,DE,6,1296
5,6,120.0,FR,-1,785
6,7,46.0,RU,0,0
7,8,28.0,CN,2,456
8,9,39.0,US,4,980
9,10,24.0,RU,7,511


In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.isna().mean()

In [7]:
data["dupl"] = data.duplicated()
data

,user_id,age,country,purchases,revenue,dupl
0,1,25.0,FR,7,749,False
1,2,24.0,RU,5,1115,False
2,3,52.0,FR,7,399,False
3,4,31.0,RU,6,654,False
4,5,NaN,DE,6,1296,False
5,6,120.0,FR,-1,785,False
6,7,46.0,RU,0,0,False
7,8,28.0,CN,2,456,False
8,9,39.0,US,4,980,False
9,10,24.0,RU,7,511,False


False


In [ ]:
data[data['dupl'] == True]

In [ ]:
data[data.duplicated(subset="user_id", keep=False)]

In [ ]:
data[ (data.age.isna()) |\
      (data.age > 100)  |\
      (data.age < 10)  |\
      (data.purchases < 0)
    ]

### Подозрительные случаи
1. Отсутствующие значения
    + `Nan` в столбце `age` 
2. Дублирующиеся значения
    + Полностью одинаковые строки с индексом 9 и 40
3. Нереалистичные значения
    + Слишком большие или маленькие значения в столбце `age`
    + Отрицательные значения в столбце `purchases`



In [ ]:
data.value_counts("country", normalize = True) #относительная частота каждой категории в столбце country


In [ ]:
data.value_counts("age", normalize = True) #заметим, что не подсчитываются отсутствующие значение

In [ ]:
result= pd.cut(data['revenue'], bins = 3, labels = ["Low", "Medium", "High"])
result

In [ ]:
bins = [0, 17, 35, 60, 120]
labels = [
    'minor',    # несовершеннолетний
    'young',    # молодой
    'mature',   # зрелый
    'elderly'   # 
]
data['age_group'] = pd.cut(data['age'], bins = bins, labels = labels)
data


In [ ]:
data["ratio"] = data["revenue"] / data["purchases"] # средний чек покупки
data

In [ ]:
group = data.groupby("country")

In [ ]:
group["ratio"].mean()

In [ ]:
data["age_group"].value_counts(normalize=True)

In [ ]:
group['age_group'].value_counts()

In [ ]:
group["purchases"].sum()

In [ ]:
group["revenue"].sum()

In [ ]:
group["revenue"].mean()

## Выводы после EDA
- Самый часто встречаемый возраст пользователей 24 (~13%)
- Больше всего пользователей из возрастной группы молодежь (~56%)
- Больше всего пользователей из России (~32%) и Франции (~29%)
- Больше всего покупок было совершенно в России (62 штуки)
- Больше всего дохода было получено в России (10 271)
- Самый высокий показатель среднего дохода с одной покупки принадлежит Китаю (232).
- А самый высокий показатель среднего дохода с одного пользователя принадлежит Германии.

In [ ]:
# Гистограмма

fig, ax = plt.subplots(  # создаем полотно
    figsize=(8, 6)  # размер полотна
)

_, _,patches = ax.hist(
    data["age"],  # данные берем из столбца с возрастом
    bins=bins,  # бины [0, 17, 30, 50, 100]
    rwidth=0.8  # столбцы с зазорами (для явного разделения возрастных групп)
)
ax.set_title("Распределение пользователей по возрастным группам")
ax.set_xlabel("Возрастная группа")
ax.set_ylabel("Количество пользователей")

plt.bar_label(patches)
    
# Добавим подписи под столбцами
bin_centers = [(bins[i] + bins[i+1]) / 2 for i in range(len(bins) - 1)]  # вычисляем центры бинов для правильных позиций меток
ax.set_xticks(bin_centers)  # ставим метки на bin_centers (в нашем случае [8.5, 23.5, 40.0, 75.0])
ax.set_xticklabels(labels)  # подписываем каждую метку с помощью labels
ax.set_yticks([0, 5, 10, 15, 20])

# Раскрасим каждый столбец
colors = plt.get_cmap('tab20c')
for i, rect in enumerate(patches):
    rect.set_fc(colors(4*i + 1))

plt.tight_layout()
plt.show() # отображаем полотна

# fig.savefig("figures/hist.png")

In [ ]:
# Ящики с усами

countries = data["country"].dropna().unique()  # получим все присутствующие в записях страны (dropna() - за исключением пустых)

fig, ax = plt.subplots(
    figsize=(8, 6)
)

ax.boxplot(
    [data[data["country"] == country]["revenue"] for country in countries],  # для каждой страны берем данные о доходе
    tick_labels=countries
)
ax.set_title('Распределение дохода по странам')
ax.set_xlabel('Страна')
ax.set_ylabel('Доход')

plt.tight_layout()
plt.show()

# fig.savefig("figures/boxplot.png")

In [ ]:
# Диграмма рассеяния

fig, ax = plt.subplots(
    figsize=(8, 6)
)

scatter = ax.scatter(
    x=data["purchases"],        # по оси X откладываем возраст
    y=data["revenue"],  # по оси Y - число покупок
    c=data["age"],  # цвет будем ранжировать по числу покупок
    cmap='plasma'         # используем такой colormap
)
cbar = plt.colorbar(scatter, ax=ax)  # отображаем цветовую карту со шкалой значений
ax.set_title('Зависимость дохода от количества покупок')
ax.set_xlabel('Число покупок')
ax.set_ylabel('Доход')
cbar.set_label("Возраст")

plt.tight_layout()
plt.show()

# fig.savefig("figures/scatter.png")